In [1]:
import ants
import tifffile
import numpy as np
import os
import cv2
import json
import time
import pandas as pd
import SimpleITK as sitk
import seaborn as sns
from scipy import stats

%matplotlib inline
import matplotlib.pyplot as plt

from multiprocessing import Pool
plt.gray()

import json, os.path, os, re, time
import joblib
import subprocess as sp
import nibabel as nib
import scipy.spatial

import sys
sys.path.append( '/home/yoshida/CUBIC-informatics/script/stitching_2023/' )
from modules.utils_220828 import *

<Figure size 432x288 with 0 Axes>

In [2]:
def fill_img(src_img,diameter):
    dilation_l_img = scipy.ndimage.morphology.grey_dilation(src_img, diameter, mode="nearest")
    erosion_l_img = scipy.ndimage.morphology.grey_erosion(dilation_l_img, diameter, mode="nearest")
    return erosion_l_img.astype(np.uint16)

In [3]:
# テキストファイルを読み込んでNumPy配列に変換する関数
def load_txt_to_numpy(file_path):
    data = {}
    with open(file_path, 'r') as file:
        for line in file:
            key, value = line.strip().split(':')
            data[key] = int(value)
    
    # NumPy配列に変換
    # キーの順番を保証するために、明示的にキーを指定して値を取得する
    numpy_array = np.array([data['z'], data['y'], data['x'], 
                            data['AngleZ'], data['AngleY'], data['AngleX']])
    return numpy_array

In [4]:
def rotate_points(points, angles):
    """
    Rotate a set of points around the X, Y, and Z axes by given angles.

    :param points: numpy array of points (shape: Nx3).
    :param angles: tuple of rotation angles (degrees) around X, Y, and Z axes.
    :return: numpy array of rotated points.
    """
    # Convert angles from degrees to radians
    angles = np.radians(angles)
    cx, cy, cz = np.cos(angles)
    sx, sy, sz = np.sin(angles)

    # Rotation matrices around X, Y, and Z axes
    Rx = np.array([[1, 0, 0], [0, cx, -sx], [0, sx, cx]])
    Ry = np.array([[cy, 0, sy], [0, 1, 0], [-sy, 0, cy]])
    Rz = np.array([[cz, -sz, 0], [sz, cz, 0], [0, 0, 1]])

    # Combined rotation matrix
    R = np.dot(Rz, np.dot(Ry, Rx))

    # Calculate centroid
    centroid = np.mean(points, axis=0)

    # Translate points to origin, rotate, and translate back
    rotated_points = np.dot(points - centroid, R.T) + centroid

    return rotated_points

In [11]:
data_parent_path = "/export2/Imaging/yoshida/AnalyzedData/OrganAtlas/"

seg_img_um = 100
seg_num = 2

actual_seg_list = [1,2]
#一番初めはatlas
sample_names = ["Liver/w8-60","Liver/w8-01","Liver/w8-72"]
seg_lists = [[0,1,2,3,4,5,6,7,8],
             [1,6,5,4,3,2,7],
             [1,2,3,4,5,6,7],]

organ_name = sample_names[0].split("/")[0]
atlas_name = sample_names[0]

In [12]:
###
def point_segmantation(PointNp2):
  PointNp = PointNp2.T
  point_np_pixel =  ((PointNp/seg_img_um)*(PointNp>0)).astype(np.uint16)
  point_np_pixel = point_np_pixel.T[point_np_pixel[2]<seg_img.shape[0]].T
  all_seg = [[PointNp.T[i][0],PointNp.T[i][1],PointNp.T[i][2],int(seg_img[point[2],point[1],point[0]])] for i,point in enumerate(point_np_pixel.T)]
  return np.array(all_seg)



In [13]:
####seg_niiに従って、seg_density_img と点の保存
for iii in [1,2]:
  sample_name = sample_names[iii]
  seg_list = seg_lists[iii]
  print(sample_name)
  density_img = tifffile.imread( os.path.join(data_parent_path,"{}/stitching/density_img_100um.tif".format(sample_name)))

  point_np = np.load( os.path.join(data_parent_path,"{}/stitching/all_points_um.npy".format(sample_name)))

  seg_img = nib.load(os.path.join(data_parent_path,"{}/segmentation/for_ants/Segmentation.nii".format(sample_name)))
  seg_img = seg_img .get_fdata().astype(np.uint8).T
  Diameter = 5
  seg_img = fill_img(seg_img,Diameter)
  seg_img = scipy.ndimage.morphology.grey_dilation(seg_img, 2, mode="nearest")

  kugiri = np.ceil(len(point_np.T)/40).astype(np.uint64)
  point_np_list = []
  for i in range(40):
    point_np_list.append(point_np.T[int(kugiri*i):int(kugiri*(i+1))])

  result = joblib.Parallel(n_jobs=10)(joblib.delayed(point_segmantation)(P) for P in point_np_list)
  Result = np.concatenate(result,0)

  for i,seg_num in enumerate(seg_list):
    each_seg_path = os.path.join(data_parent_path,"{}/segmentation/seg{}/".format(sample_name,str(i).zfill(3)))
    #print(each_seg_path)
    make_dir(each_seg_path)
    Ps = Result[Result.T[3] == seg_num]
    seg_d_img = make_density_image(Ps.T[0]/seg_img_um,Ps.T[1]/seg_img_um,Ps.T[2]/seg_img_um,\
                      density_img.shape[2],density_img.shape[1],density_img.shape[0])
    tifffile.imsave(os.path.join(each_seg_path,"seg_density_img.tif"),seg_d_img.astype(np.uint16))
    np.save(os.path.join(each_seg_path,"all_points_um.npy"),Ps[:,:3])

Liver/w8-01
Liver/w8-72


# ANTs

In [15]:
class mapping_to_atlas():
    def __init__(self, sample_name,seg_num):
        self.ants_voxel_unit = seg_img_um
        self.original_voxel_unit = seg_img_um
        self.img_voxel_unit = seg_img_um

        
        self.atlas_tif_path = os.path.join(data_parent_path,"{}/segmentation/{}/seg_density_img.tif".format(atlas_name,seg_num))
        self.atlas_nii_path = self.atlas_tif_path.replace(".tif",".nii.gz")
        
        #for image size check
        self.sample_density_img_path = os.path.join(data_parent_path,"{}/segmentation/{}/seg_density_img.tif".format(sample_name,seg_num))
        
        self.flip_tif_path =  os.path.join(data_parent_path,"{}/segmentation/{}/seg_density_img_flip.tif".format(sample_name,seg_num))
        self.flip_txt_path = os.path.join(data_parent_path,"{}/segmentation/{}/flip_memo.txt".format(sample_name,seg_num))

        self.ants_dst_dir = os.path.join(data_parent_path,"{}/segmentation/{}/".format(sample_name,seg_num))
        
        #self.flip_tif_path --->  self.sample_nii_path
        self.sample_nii_path = os.path.join(self.ants_dst_dir ,"before_ants.nii.gz")
        
        self.before_ants_np_path = os.path.join(data_parent_path,"{}/segmentation/{}/all_points_um.npy".format(sample_name,seg_num))
        
        if not os.path.exists(self.ants_dst_dir):
            print("There is no ants folder!!!")

        self.moving_nii_path = self.sample_nii_path
        self.prefix_ants = "/usr/local/ants-2.1.0-redhat/"
        
        self.moving_points_path = os.path.join(self.ants_dst_dir,"moving.csv")
        self.moved_points_path = os.path.join(self.ants_dst_dir,"moved.csv")
        self.moved_denstity_path = os.path.join(self.ants_dst_dir,"moved_denstity.tif")
    
        
    def tif2nii(self,tif_path,nii_path,nii_voxel_unit):
        tif_img = tifffile.imread(tif_path)
        nii_img = nib.Nifti1Image(np.swapaxes(tif_img,0,2), affine=None)
        aff = np.diag([-nii_voxel_unit,-nii_voxel_unit,nii_voxel_unit,1])
        nii_img.header.set_qform(aff, code=2)
        nii_img.to_filename(nii_path)
        return


    def run_antsRegistration(self,prefix_ants, atlas_file, moving_file, dst_dir, threads):
        cmd = "ITK_GLOBAL_DEFAULT_NUMBER_OF_THREADS={THREADS} && "
        cmd += "{EXECUTABLE} -d 3 "
        cmd += "--initial-moving-transform [{ATLAS_FILE},{MOVING_FILE},1] "
        cmd += "--interpolation Linear "
        cmd += "--use-histogram-matching 0 "
        cmd += "--winsorize-image-intensities [0.05,1.0] "  
        cmd += "--float 0 "
        cmd += "--output [{DST_PREFIX},{WARPED_FILE},{INVWARPED_FILE}] "
        cmd += "--transform Affine[0.1] --metric MI[{ATLAS_FILE},{MOVING_FILE},1,128,Regular,0.5] --convergence [10000x10000x10000,1e-5,15] --shrink-factors 8x4x2 --smoothing-sigmas 3x2x1vox "
        cmd += "--transform SyN[0.1,3.0,0.0] --metric CC[{ATLAS_FILE},{MOVING_FILE},1,4] --convergence [500x500x500x50,1e-6,10] --shrink-factors 8x4x2x1 --smoothing-sigmas 3x2x1x0vox"
        cmd = cmd.format(
            THREADS = threads,
            EXECUTABLE = os.path.join(prefix_ants, "antsRegistration"),
            DST_PREFIX = os.path.join(dst_dir, "F2M_"),
            WARPED_FILE = os.path.join(dst_dir, "F2M_Warped.nii.gz"),
            INVWARPED_FILE = os.path.join(dst_dir, "F2M_InvWarped.nii.gz"),
            ATLAS_FILE = atlas_file,
            MOVING_FILE = moving_file,
        )
        print("[*] Executing : {}".format(cmd))
        sp.call(cmd, shell=True)
        return

    #scale of list is atlas_voxel_unit
    def make_density_image(self,list_x,list_y,list_z,size_x,size_y,size_z):
        depth,height,width = size_z,size_y,size_x
        density_img,_ = np.histogramdd(
            np.vstack([list_z,list_y,list_x]).T,
            bins=(depth, height, width),
            range=[(0,depth-1),(0,height-1),(0,width-1)]
        )
        return density_img
    
    def run_antsApplyTransformsToPoints(self, prefix_ants, src_csv, dst_csv, ANTs_image_dir):
        cmd = "{EXECUTABLE} "
        cmd += "-d 3 "
        cmd += "-i {SRC_CSV} "
        cmd += "-o {DST_CSV} "
        cmd += "-t [{AFFINE_MAT},1] "
        cmd += "-t {INVWARP_NII}"
        cmd = cmd.format(
            EXECUTABLE = os.path.join(prefix_ants, "antsApplyTransformsToPoints"),
            AFFINE_MAT = os.path.join(ANTs_image_dir, "F2M_0GenericAffine.mat"),
            INVWARP_NII = os.path.join(ANTs_image_dir, "F2M_1InverseWarp.nii.gz"),
            SRC_CSV = src_csv,
            DST_CSV = dst_csv,
        )
        #print("[*] Executing : {}".format(cmd))
        # supress output
        with open(os.devnull, 'w') as devnull:
            sp.check_call(cmd, shell=True, stdout=devnull)
        return